# Word Sense Disambiguation using Neural Networks
Adam Ek

----------------------------------

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).

Write all your answers and the code in the appropriate boxes below.

----------------------------------

A problem with static distributional vectors is the difficulty of distinguishing between different *word senses*. We will continue our exploration of word vectors by considering *trainable vectors* or *word embeddings* for Word Sense Disambiguation (WSD).

The goal of word sense disambiguation is to train a model to find the sense of a word (homonyms of a word-form). For example, the word "bank" can mean "sloping land" or "financial institution". 

(a) "I deposited my money in the **bank**" (financial institution)

(b) "I swam from the river **bank**" (sloping land)

In case a) and b) we can determine that the meaning of "bank" based on the *context*. To utilize context in a semantic model we use *contextualized word representations*. Previously we worked with *static word representations*, i.e. the representation does not depend on the context. To illustrate we can consider sentences (a) and (b), the word **bank** would have the same static representation in both sentences, which means that it becomes difficult for us to predict its sense. What we want is to create representations that depend on the context, i.e. *contextualized embeddings*. 

We will create contextualized embeddings with Recurrent Neural Networks. You can read more about recurrent neural netoworks [here](https://colah.github.io/posts/2015-08-Understanding-LSTMs/). Your overall task in this lab is to create a neural network model that can disambiguate the word sense of 30 different words. 

In [56]:
# first we import some packages that we need
import torch
import torch.nn as nn
import torchtext
import random
import torch.nn.functional as F
import torch.optim as optim
from torchtext.data import Field, BucketIterator, Iterator, TabularDataset
from IPython import embed
import numpy as np

# our hyperparameters (add more when/if you need them)
device = torch.device('cpu')
hyperparameters = {
'batch_size' : 8,
'learning_rate' : 0.001,
'epochs' : 3,
'embedding_dim' : 32,
'output_dim' : 32
}

# 1. Working with data

A central part of any machine learning system is the data we're working with. In this section we will split the data (the dataset is located here: ``wsd-data/wsd_data.txt``) into a training set and a test set. We will also create a baseline to compare our model against. Finally, we will use TorchText to transform our data (raw text) into a convenient format that our neural network can work with.

## Data

The dataset we will use contain different word sense for 30 different words. The data is organized as follows (values separated by tabs): 
- Column 1: word-sense
- Column 2: word-form
- Column 3: index of word
- Column 4: white-space tokenized context

### Splitting the data

Your first task is to seperate the data into a *training set* and a *test set*. The training set should contain 80% of the examples and the test set the remaining 20%. The examples for the test/training set should be selected **randomly**. Save each dataset into a .csv file for loading later. **[2 marks]**

In [57]:
def data_split(path_to_dataset):
        
    with open(path_to_dataset) as f:
        dataset = [line for line in f]
        random.shuffle(dataset)
    
    train = dataset[:int(len(dataset)*0.8)] 
    test = dataset[-int(len(dataset)*0.2):] 
    
    with open('train.csv', 'w') as f:
        for line in train:
            f.write(line)
    
    with open('test.csv', 'w') as f:
        for line in test:
            f.write(line)
    
    return

data_split('wsd-data/wsd_data.txt')

### Creating a baseline

Your second task is to create a *baseline* for the task. A baseline is a "reality check" for a model, given a very simple heuristic/algorithmic/model solution to the problem, can our neural network perform better than this?
The baseline you are to create is the "most common sense" (MCS) baseline. For each word form, find the most commonly assigned sense to the word, and label a words with that sense. **[2 marks]**

E.g. In a fictional dataset, "bank" have two senses, "financial institution" which occur 5 times and "side of river" 3 times. Thus, all 8 occurences of bank is labeled "financial institution" and this yields an MCS accuracy of 5/8 = 62.5%. If a model obtain a higher score than this, we can conclude that the model *at least* is better than selecting the most frequent word sense.

In [58]:
def mcs_baseline(data):
    
    dataset = []
    with open(data) as f:
        for line in f:
            token = line.strip('\n')
            row = token.split('\t')
            dataset.append(row)
    
    word_forms_unique = set([row[1] for row in dataset])
    word_forms_dict = dict.fromkeys(word_forms_unique, 0)
    
    for word_form in word_forms_unique:
        d = {}
        for row in dataset:
            if row[1] == word_form:
                if row[0] not in d:
                    d[row[0]] = 1
                else:
                    d[row[0]] += 1
        word_forms_dict[word_form] = d
    
    baseline_dict = dict.fromkeys(word_forms_unique, 0)
    for word_form, word_sense_dict in word_forms_dict.items():
        total = []
        for word_sense, count in word_sense_dict.items():
            total.append(count)
        baseline_dict[word_form] = (max(total)/sum(total))
    
    return baseline_dict

baseline_dict = mcs_baseline('wsd-data/wsd_data.txt')
print(baseline_dict)

{'build.v': 0.18787878787878787, 'life.n': 0.2366412213740458, 'security.n': 0.2206896551724138, 'serve.v': 0.19642857142857142, 'positive.a': 0.2987012987012987, 'professional.a': 0.2857142857142857, 'place.n': 0.2909090909090909, 'line.n': 0.8192419825072886, 'position.n': 0.23529411764705882, 'physical.a': 0.3484848484848485, 'major.a': 0.36046511627906974, 'active.a': 0.39759036144578314, 'follow.v': 0.17560975609756097, 'bring.v': 0.2375, 'common.a': 0.3153153153153153, 'lead.v': 0.20134228187919462, 'point.n': 0.29464285714285715, 'order.n': 0.24347826086956523, 'case.n': 0.23684210526315788, 'regular.a': 0.2037037037037037, 'national.a': 0.24647887323943662, 'force.n': 0.19161676646706588, 'time.n': 0.2585034013605442, 'critical.a': 0.3466666666666667, 'bad.a': 0.6391752577319587, 'keep.v': 0.38461538461538464, 'see.v': 0.6615384615384615, 'find.v': 0.25547445255474455, 'extend.v': 0.2, 'hold.v': 0.18134715025906736}


### Creating data iterators

To train a neural network, we first need to prepare the data. This involves converting words (and labels) to a number, and organizing the data into batches. We also want the ability to shuffle the examples such that they appear in a random order.  

To do all of this we will use the torchtext library (https://torchtext.readthedocs.io/en/latest/index.html). In addition to converting our data into numerical form and creating batches, it will generate a word and label vocabulary, and data iterators than can sort and shuffle the examples. 

Your task is to create a dataloader for the training and test set you created previously. So, how do we go about doing this?

1) First we create a ``Field`` for each of our columns. A field is a function which tokenize the input, keep a dictionary of word-to-numbers, and fix paddings. So, we need four fields, one for the word-sense, one for the position, one for the lemma and one for the context. 

2) After we have our fields, we need to process the data. For this we use the ``TabularDataset`` class. We pass the name and path of the training and test files we created previously, then we assign which field to use in each column. The result is that each column will be processed by the field indicated. So, the context column will be tokenized and processed by the context field and so on. 

3) After we have processed the dataset we need to build the vocabulary, for this we call the function ``build_vocab()`` on the different ``Fields`` with the output from ``TabularDataset`` as input. This looks at our dataset and creates the necessary vocabularies (word-to-number mappings). 

4) Finally, the last step. In the last step we load the data objects given by the ``TabularDataset`` and pass it to the ``BucketIterator`` class. This class will organize our examples into batches and shuffle them around (such that for each epoch the model observe the examples in a different order). When we are done with this we can let our function return the data iterators and vocabularies, then we are ready to train and test our model!

Implement the dataloader. [**2 marks**]

*hint: for TabularDataset and BucketIterator use the class function splits()* 

In [59]:
def dataloader(path,batchsize):

    whitespacer = lambda x: x.split(' ')
    
    WORDSENSE = Field(tokenize    = whitespacer,
                   lower       = True,
                   batch_first = True) # enforce the (batch, words) structure
    LEMMA = Field(tokenize    = whitespacer,
                   lower       = True,
                   batch_first = True)
    POSITION = Field(tokenize    = whitespacer,
                   lower       = True,
                   batch_first = True)
    CONTEXT = Field(tokenize    = whitespacer,
                   lower       = True,
                   batch_first = True) 

    train,test = TabularDataset.splits(path   = path,
                                       train = 'train.csv',
                                       test = 'test.csv',
                            format = 'csv',
                            fields = [('wordsense', WORDSENSE),
                                    ('lemma', LEMMA),
                                     ('position', POSITION),
                                     ('context', CONTEXT)],
                            skip_header       = True,
                            csv_reader_params = {'delimiter':'\t',
                                                             'quotechar':'½'})
    # build vocabularies based on what our csv files contained and create word2id mapping
    WORDSENSE.build_vocab(train)
    LEMMA.build_vocab(train)
    POSITION.build_vocab(train)
    CONTEXT.build_vocab(train)
    
    # create batches from our data, and shuffle them for each epoch
    train_iter, test_iter = BucketIterator.splits((train,test),
                                batch_size        = batchsize,
                                sort_within_batch = True,
                                sort_key          = lambda x: len(x.context),
                                shuffle           = True,
                                device            = device)
    
    return train_iter, test_iter, WORDSENSE, LEMMA, POSITION, CONTEXT

train_iter, test_iter, WORDSENSE, LEMMA, POSITION, CONTEXT = dataloader('.', hyperparameters['batch_size'])

In [41]:
print(CONTEXT.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fd9d4535b50>>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'of': 3, ',': 4, 'and': 5, 'to': 6, '.': 7, 'in': 8, 'a': 9, 'that': 10, 'for': 11, 'on': 12, 'with': 13, 'as': 14, 'is': 15, '(': 16, ')': 17, 'by': 18, 'be': 19, 'it': 20, 'or': 21, 'its': 22, 'was': 23, 'at': 24, 'are': 25, 'which': 26, ';': 27, 'united': 28, 'not': 29, 'an': 30, 'this': 31, "'s": 32, 'international': 33, 'from': 34, 'their': 35, 'have': 36, 'nations': 37, 'has': 38, 'all': 39, 'security': 40, 'also': 41, 'been': 42, 'had': 43, 'we': 44, 'rights': 45, 'other': 46, 'states': 47, 'were': 48, 'development': 49, 'would': 50, 'committee': 51, 'should': 52, 'such': 53, 'will': 54, 'countries': 55, '“': 56, 'council': 57, 'national': 58, 'women': 59, '”': 60, ':': 61, 'government': 62, 'human': 63, 'law': 64, 'state': 65, 'under': 66, 'his': 67, 'they': 68, 'more': 69, 'order': 70, 'those': 71, 'general': 72, 'work': 73, 'i': 74, 'may'

# 2.1 Creating and running a Neural Network for WSD

In this section we will create and run a neural network to predict word senses based on *contextualized representations*.

### Model

We will use a bidirectional Long-Short-Term Memory (LSTM) network to create a representation for the sentences and a Linear classifier to predict the sense of each word.

When we initialize the model, we need a few things:

    1) An embedding layer: a dictionary from which we can obtain word embeddings
    2) A LSTM-module to obtain contextual representations
    3) A classifier that compute scores for each word-sense given *some* input


The general procedure is the following:

    1) For each word in the sentence, obtain word embeddings
    2) Run the embedded sentences through the RNN
    3) Select the appropriate hidden state
    4) Predict the word-sense 

**Suggestion for efficiency:**  *Use a low dimensionality (32) for word embeddings and the LSTM when developing and testing the code, then scale up when running the full training/tests*
    
Your tasks will be to create two different models (both follow the two outlines described above), described below:

In the first approach to WSD, you are to select the index of our target word (column 3 in the dataset) and predict the word sense. **[5 marks]**


In [79]:
class WSDModel_approach1(nn.Module):
    def __init__(self, vocab_size, wordsense_size, emb_dim, output_dim):
        super(WSDModel_approach1, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, emb_dim)
        self.rnn = nn.LSTM(emb_dim, output_dim)
        self.classifier = nn.Linear(output_dim, wordsense_size)
    
    def forward(self, batch):
        embedded_batch = self.embeddings(batch)
        embedded_batch_2 = self.rnn(embedded_batch)
        predicted_sense = self.classifier(embedded_batch_2)
        
        return predicted_sense

In the second approach to WSD, you are to predict the word sense based on the final hidden state given by the RNN. **[5 marks]**

In [ ]:
class WSDModel_approach2(nn.Module):
    def __init__(self, vocab_size, wordsense_size, emb_dim, output_dim):
        self.embeddings = nn.Embedding(vocab_size, emb_dim)
        self.rnn = nn.LSTM(emb_dim, output_size)
        self.classifier = nn.Linear(output_size, wordsense_size)
    
    def forward(self, batch):
        
        
        return predictions

### Training and testing the model

Now we are ready to train and test our model. What we need now is a loss function, an optimizer, and our data. 

- First, create the loss function and the optimizer.
- Next, we iterate over the number of epochs (i.e. how many times we let the model see our data). 
- For each epoch, iterate over the dataset (``train_iter``) to obtain batches. Use the batch as input to the model, and let the model output scores for the different word senses.
- For each model output, calculate the loss (and print the loss) on the output and update the model parameters.
- Reset the gradients and repeat.
- After all epochs are done, test your trained model on the test set (``test_iter``) and calculate the total and per-word-form accuracy of your model.

Implement the training and testing of the model **[4 marks]**

**Suggestion for efficiency:** *when developing your model, try training and testing the model on one or two batches (for each epoch) of data to make sure everything works! It's very annoying if you train for N epochs to find out that something went wrong when testing the model, or to find that something goes wrong when moving from epoch 0 to epoch 1.*

In [84]:
train_iter, test_iter, WORDSENSE, LEMMA, POSITION, CONTEXT = dataloader('.', batch_size)

loss_function = nn.CrossEntropyLoss()

vocab_size = len(CONTEXT.vocab)
num_labels = len(WORDSENSE.vocab)

model = WSDModel_approach1(
        vocab_size,
        num_labels,
        hyperparameters['embedding_dim'],
        hyperparameters['output_dim'])


optimizer = optim.Adam(
            model.parameters(),
            lr=hyperparameters['learning_rate'])

for epoch in range(epochs):
    for i, batch in enumerate(train_iter):
        _context = batch.context
        _word_sense = batch.wordsense
        
        gold_data = _word_sense
        
        output = model(_context)

TypeError: linear(): argument 'input' (position 1) must be Tensor, not tuple

# 2.2 Running a transformer for WSD

In this section of the lab you'll try out the transformer, specifically the BERT model. For this we'll use the huggingface library (https://huggingface.co/).

You can find the documentation for the BERT model here (https://huggingface.co/transformers/model_doc/bert.html) and a general usage guide here (https://huggingface.co/transformers/quickstart.html).

What we're going to do is *fine-tune* the BERT model, i.e. update the weights of a pre-trained model. That is, we have a model that is trained on language modeling, but now we apply it to word sense disambiguation with the word representations it learnt from language modeling.

We'll use the same data splits for training and testing as before, but this time you'll not use a torchtext dataloader. Rather now you create an iterator that collects N sentences (where N is the batch size) then use the BertTokenizer to transform the sentence into integers. For your dataloader, remember to:
* Shuffle the data in each batch
* Make sure you get a new iterator for each *epoch*
* Create a vocabulary of *sense-labels* so you can calculate accuracy 

We then pass this batch into the BERT model and train as before. The BERT model will encode the sentence, then we send this encoded sentence into a prediction layer (you can either the the sentence-representation from bert, or the ambiguous word) like before and collect sense predictions.

About the hyperparameters and training:
* For BERT, usually a lower learning rate works best, between 0.0001-0.000001.
* BERT takes alot of resources, running it on CPU will take ages, utilize the GPUs :)
* Since BERT takes alot of resources, use a small batch size (4-8)
* Computing the BERT representation, make sure you pass the mask

**[10 marks]**

In [ ]:
def dataloader_for_bert(path_to_file, batch_size):
    ...

In [ ]:
class BERT_WSD(nn.Module):
    def __init__(self, ...):
        # your code goes here
        self.bert = ...
        self.classifier = ...
    
    def forward(self, batch):
        # your code goes here
        
        return predictions

In [ ]:
loss_function = ...
optimizer = ...
model = ...

for _ in range(epochs):
    # train model
    ...
    
# test model after all epochs are completed

# 3. Evaluation

Explain the difference between the first and second approach. What kind of representations are the different approaches using to predict word-senses? **[4 marks]**

Evaluate your model with per-word-form *accuracy* and comment on the results you get, how does the model perform in comparison to the baseline, and how do the models compare to each other? 

Expand on the evaluation by sorting the word-forms by the number of senses they have. Are word-forms with fewer senses easier to predict? Give a short explanation of the results you get based on the number of senses per word.

**[6 marks]**

How does the LSTMs perform in comparison to BERT? What's the difference between representations obtained by the LSTMs and BERT? **[2 marks]**

What could we do to improve our LSTM word sense disambiguation models and our BERT model? **[4 marks]**

# Readings:

[1] Kågebäck, M., & Salomonsson, H. (2016). Word Sense Disambiguation using a Bidirectional LSTM. arXiv preprint arXiv:1606.03568.

[2] https://cl.lingfil.uu.se/~nivre/master/NLP-LexSem.pdf